In [1]:
import networkx as nx
import random
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os

from IPython.display import HTML
from GraphGenusGenerator import GraphGenusGenerator
from HC_Algorithm import hill_climb
from SimulatedAnnealing import simulated_annealing
from RHC_Algorithm import random_hill_climb
from Genetic_Algorithm import Genetic_Algorithm
from mlrose_hiive.algorithms.decay import GeomDecay
from mlrose_hiive.algorithms.decay import ArithDecay
from mlrose_hiive.algorithms.decay import ExpDecay

In [2]:
def is_power_of_two(n):
    return (n & (n - 1) == 0) and n != 0

def get_best_result(file_path):
    df = pd.read_csv(file_path)
    min_fitness = df['Fitness'].min()
    first_min_fitness_index = df[df['Fitness'] == min_fitness].index[0]
    best_result = df.loc[first_min_fitness_index]
    return best_result

def save_best_result(best_result, number_of_nodes, bracket, graph_type, seed, algorithm_name, listed_fitness_over_time, file_path="all_results.csv"):
    if graph_type == "billiards":
        data = {
            'Algorithm': [algorithm_name],
            'Graph Type': [graph_type],
            'Bracket': [bracket],
            'Final Fitness': [best_result['Fitness']],
            'Plato': [best_result['Iteration']],
            'Time': [best_result['Time']],
            'Seed': [seed],
            'Fitness List': [listed_fitness_over_time],
            'Final State': [best_result['State']]
        }
    else:
        data = {
            'Algorithm': [algorithm_name],
            'Graph Type': [graph_type],
            'Number of Nodes': [number_of_nodes],
            'Final Fitness': [best_result['Fitness']],
            'Plato': [best_result['Iteration']],
            'Time': [best_result['Time']],
            'Seed': [seed],
            'Fitness List': [listed_fitness_over_time],
            'Final State': [best_result['State']]
        }
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    if os.path.exists(file_path):
        df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        df.to_csv(file_path, index=False)

def generate_seeds(original_seed, count):
    random.seed(original_seed)
    return [random.randint(0, 1000000) for _ in range(count)]

def get_fitness_over_time(filtered_data):
    fitness_values = filtered_data['Fitness'].tolist()
    return fitness_values

def random_hill_climb_get_fitness_over_time(iteration_data):
    power = 0
    FitnessList = iteration_data['Fitness'][1:]
    iter_length = len(FitnessList)
    best_fit = FitnessList.iloc[0]

    best_fit_list = []
    for i in range(iter_length):
        current_fitness = FitnessList.iloc[i]
        if current_fitness < best_fit:
            best_fit = current_fitness
        if i == 2**power:
            best_fit_list.append(best_fit)
            power += 1

    return best_fit_list

# Main code
result_file = "all_results.csv"
bracket=None

graph_types = ["complete", "complete_bipartite"]
algorithms = ["Simulated_Annealing", "Genetic"]
seed = random.randint(0, 2**32 - 1)
times_to_run = 100
max_iter = np.inf

for graph in graph_types:
    match graph:
        case "complete":
            problem = GraphGenusGenerator.generate(seed=seed, number_of_nodes=8, graph_type="complete")    
        case "complete_bipartite":
            problem = GraphGenusGenerator.generate(seed=seed, number_of_nodes=10, graph_type="complete_bipartite")
        case "billiards":
            bracket=[1,3,1,2]
            problem = GraphGenusGenerator.generate(seed=seed, graph_type="billiards", bracket=bracket)  
        case _:
            print("graph_name undefined")
            break

    for algorithm in algorithms:
        match algorithm:
            case "Simulated_Annealing":
                decay_rates = [0.9999, 0.999, 0.995, 0.99, 0.95, 0.9, 0.85]
                init_temps = [5, 8, 10, 12, 15, 18, 20]
                for temp in init_temps:
                    for rate in decay_rates:
                        new_seeds = generate_seeds(seed, times_to_run)
                        for i in range(times_to_run):
                            record_file = f"{algorithm}_iterations.csv"
                            new_seed = new_seeds[i]
                            problem.random()
                            final_state, final_fitness, iteration_data = simulated_annealing(problem=problem, schedule=GeomDecay(init_temp=temp, decay=rate, min_temp=0.001), max_iters=max_iter, max_attempts=50, random_state=new_seed, curve=True)
                            
                            df_run_stats = pd.DataFrame(iteration_data)
                            df_run_stats.to_csv(record_file, index=False)

                            filtered_data = df_run_stats[(df_run_stats['Iteration'] == 0) | 
                                                         (df_run_stats['Iteration'].apply(is_power_of_two)) | 
                                                         (df_run_stats['Iteration'] == df_run_stats['Iteration'].max())]
                            listed_fitness_over_time = get_fitness_over_time(filtered_data)

                            best_result = get_best_result(record_file)
                            save_best_result(best_result, problem.length, bracket, graph, new_seed, algorithm, listed_fitness_over_time, file_path=f"NewData/{algorithm}_{graph}_{temp}/{rate}_Best_Results_Summary.csv")

                            print(f"Run {i+1}/{times_to_run}")
                            print(listed_fitness_over_time)
                            print("Final Fitness:", final_fitness)
                            print("Final State:", final_state)
                            print("Plato", best_result['Iteration'])

            case "Genetic":
                pop_sizes = [25, 50, 100, 150, 200, 250]
                mutation_probs = [0.05, 0.08, 0.1, 0.13, 0.15, 0.18, 0.2]

                for pop_size in pop_sizes:
                    for prob in mutation_probs:
                        new_seeds = generate_seeds(seed, times_to_run)
                        for i in range(times_to_run):
                            record_file = f"{algorithm}_iterations.csv"
                            new_seed = new_seeds[i]
                            problem.random()
                            final_fitness, final_state, iteration_data = Genetic_Algorithm(problem=problem, max_attempts=2000, max_iters=max_iter, restarts=0, pop_size=pop_size, pop_breed_percent=0.98, elite_dreg_ratio=0.85, minimum_elites=2, minimum_dregs=0, mutation_prob=prob, record_file=None)
                            
                            df_run_stats = pd.DataFrame(iteration_data)
                            df_run_stats.to_csv(record_file, index=False)

                            filtered_data = df_run_stats[(df_run_stats['Iteration'] == 0) | 
                                                         (df_run_stats['Iteration'].apply(is_power_of_two)) | 
                                                         (df_run_stats['Iteration'] == df_run_stats['Iteration'].max())]
                            listed_fitness_over_time = get_fitness_over_time(filtered_data)

                            best_result = get_best_result(record_file)
                            save_best_result(best_result, problem.length, bracket, graph, new_seed, algorithm, listed_fitness_over_time, file_path=f"NewData/{algorithm}_{graph}_{pop_size}/{prob}_Best_Results_Summary.csv")

                            print(f"Run {i+1}/{times_to_run}")
                            print(listed_fitness_over_time)
                            print("Final Fitness:", final_fitness)
                            print("Final State:", final_state)
                            print("Plato", best_result['Iteration'])

            case _:
                print("algorithm_name undefined")
                break


Run 1/100
[10.0, 9.0, 8.0, 8.0, 10.0, 7.0, 9.0, 10.0, 10.0, 9.0, 9.0, 10.0, 10.0, 9.0, 9.0, 7.0]
Final Fitness: 7.0
Final State: {0: [1, 3, 6, 7, 5, 4, 2], 1: [0, 4, 6, 7, 3, 2, 5], 2: [0, 4, 7, 5, 6, 1, 3], 3: [0, 2, 1, 6, 4, 5, 7], 4: [0, 5, 6, 7, 1, 3, 2], 5: [0, 4, 6, 3, 2, 7, 1], 6: [0, 1, 4, 7, 3, 5, 2], 7: [0, 4, 5, 3, 1, 2, 6]}
Plato 7575
Run 2/100
[9.0, 9.0, 8.0, 10.0, 10.0, 8.0, 9.0, 9.0, 9.0, 8.0, 9.0, 9.0, 10.0, 9.0, 8.0, 6.0]
Final Fitness: 6.0
Final State: {0: [1, 4, 6, 7, 2, 5, 3], 1: [0, 2, 7, 5, 6, 4, 3], 2: [0, 7, 3, 4, 6, 1, 5], 3: [0, 6, 7, 4, 1, 2, 5], 4: [0, 3, 6, 5, 2, 1, 7], 5: [0, 2, 3, 7, 4, 1, 6], 6: [0, 7, 1, 5, 3, 4, 2], 7: [0, 4, 1, 3, 6, 5, 2]}
Plato 12511
Run 3/100
[10.0, 10.0, 10.0, 9.0, 10.0, 10.0, 9.0, 9.0, 9.0, 10.0, 10.0, 10.0, 10.0, 9.0, 9.0, 7.0]
Final Fitness: 7.0
Final State: {0: [1, 2, 5, 6, 4, 7, 3], 1: [0, 6, 7, 3, 5, 4, 2], 2: [0, 6, 3, 4, 1, 5, 7], 3: [0, 4, 1, 6, 7, 5, 2], 4: [0, 6, 5, 1, 3, 2, 7], 5: [0, 1, 7, 3, 6, 2, 4], 6: [0, 4, 2, 5,

KeyboardInterrupt: 